In [45]:
import pandas as pd
import numpy as np
import os
import tqdm
import shutil

In [46]:
dataset = 'onion' # onion or emma
input_data_dir = f'../data/'

features = {
    'emotion_embeddings': ['id_emma.tsv', 'id_gems.tsv'],
    #'audio_embeddings': ['id_maest.tsv'],
    #'textual_embeddings': ['id_bert.tsv'],
    #'visual_embeddings': ['id_resnet.tsv']
}

output_dir = f'../dataset/{dataset}/'

os.makedirs(output_dir, exist_ok=True)

In [47]:
df_interactions = pd.read_csv(f'../data/userid_trackid_timestamp_{dataset}.tsv', sep='\t')
    
df_interactions['rating'] = 5
sample_items = df_interactions['track_id'].unique()

# map item and users to numbers
df_interactions['user_id_int'] = df_interactions['user_id'].astype('category').cat.codes
df_interactions['track_id_int'] = df_interactions['track_id'].astype('category').cat.codes


print(df_interactions.shape)
df_interactions.head()

(1000000, 6)


,user_id,track_id,timestamp,rating,user_id_int,track_id_int
0,51549,iJTBIGHPjgJcT4Bt,2013-01-27 21:42:38,5,724,21689
1,51549,iJTBIGHPjgJcT4Bt,2013-01-27 21:38:53,5,724,21689
2,51549,iJTBIGHPjgJcT4Bt,2013-01-27 21:35:08,5,724,21689
3,51549,iJTBIGHPjgJcT4Bt,2013-01-27 21:31:23,5,724,21689
4,51549,iJTBIGHPjgJcT4Bt,2013-01-27 21:27:38,5,724,21689


In [48]:
# track ids from feature file
df = pd.read_csv(input_data_dir + features['emotion_embeddings'][1], sep='\t')
feat_track_ids = df['id'].values
feat_track_ids

array(['04q3VppIQEET5rzy', '07xF9Q0K1t3ist7K', '0JbYcELIqpMPvAHk', ...,
       'zzyyPUs7hC9Nz2e1', 'zzz0n04uuTUA7fNh', 'zzzj3LYaZtYtbzSr'],
      dtype=object)

In [49]:
# remove interactions with items not in the feature files
df_interactions = df_interactions[df_interactions['track_id'].isin(feat_track_ids)]
df_interactions.shape

(999985, 6)

In [50]:
#df_interactions[['user_id', 'track_id', 'rating', 'timestamp']].to_csv(os.path.join(output_dir, 'interactions.tsv'), index=False, sep='\t', header=False)
df_interactions[['user_id_int', 'track_id_int', 'rating', 'timestamp']].to_csv(os.path.join(output_dir, 'interactions.tsv'), index=False, sep='\t', header=False)

In [51]:
item_id_map = df_interactions[['track_id', 'track_id_int']].drop_duplicates()
item_id_map.set_index('track_id', inplace=True)
item_id_map.head()

,track_id_int
track_id,
iJTBIGHPjgJcT4Bt,21689
LCItxaUrpHk6QYuy,10397
VXVSlV3nA5jgYOW1,15482
DQ9EMmQndbcKKbBo,6670
d6hSPGsvbBx2mcPR,19113


In [52]:
del df_interactions

In [53]:
for feature_type in features:

    for file in features[feature_type]:
        if dataset == 'onion' and 'emma' in file:
            continue

        print('Processing', file)
        df = pd.read_csv(os.path.join(input_data_dir, file), index_col=0, sep='\t')
        df = df.merge(item_id_map, left_index=True, right_index=True)
        feature_name = file.split('.')[0].split('_')[1]
            
        folder_path = os.path.join(output_dir, feature_type, feature_name)

        if os.path.exists(folder_path):
            shutil.rmtree(folder_path)
        os.makedirs(folder_path, exist_ok=True)

        for index, row in tqdm.tqdm(df.iterrows()):
            np.save(os.path.join(folder_path, str(int(row['track_id_int'])) + '.npy'), row.values[:-1])

Processing id_gems.tsv


30518it [00:03, 8305.21it/s]
